## abusing을 결정짓는 feature importance를 알아보기 위해 Decision Tree 이용

In [1]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split # 알아서 테스트랑 트래인 잘라주는거

In [27]:
from sklearn.preprocessing import minmax_scale

In [29]:
import numpy as np

In [61]:
train = pd.read_csv('new_melon1819_unique.csv')

In [62]:
train.sample()

,song_id,title,like,reply,genre,artist_id,artist,fan,album_id,album,...,album_reply,album_score,album_score_count,abusing,count,fan_max,user_per,date,date_range,album_like
641,31864628,BIRTHDAY,54294.0,736.0,['댄스'],[968260],['전소미'],[21736],10296482.0,BIRTHDAY,...,6552.0,3.6,11320.0,0,10,21736,1.994475,2019-06-10,176,9858.0


In [63]:
train['album_like/like'] = train['album_like']/train['like']

In [64]:
train['fan_per'] = minmax_scale(np.array(train['fan_max']), axis=0, copy=True)

In [65]:
train['album_reply_per'] = minmax_scale(np.array(train['album_reply']), axis=0, copy=True)
train['reply_per'] = minmax_scale(np.array(train['reply']), axis=0, copy=True)
train['like_per'] = minmax_scale(np.array(train['like']), axis=0, copy=True)

In [66]:
train['album_score_per'] = minmax_scale(np.array(train['album_score']), axis=0, copy=True)
train['album_score_count_per'] = minmax_scale(np.array(train['album_score_count']), axis=0, copy=True)
train['user_per_per'] = minmax_scale(np.array(train['user_per']), axis=0, copy=True)

In [67]:
train['album_like_per'] = minmax_scale(np.array(train['album_like']), axis=0, copy=True)
train['album_like_like_per'] = minmax_scale(np.array(train['album_like/like']), axis=0, copy=True)

### Split train validation

In [68]:
features = ['fan_per', 'like_per', 'album_score_per', 'album_like_like_per']

In [69]:
X = train[features]
y = train['abusing']

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # 섞어서 알아서 0.2비율로 만들어줌
# random_sate를 정해주지 않으면 항상 달라져서 42로 줬다.

### Make classification model

In [71]:
tree = DecisionTreeClassifier(random_state=42)

In [72]:
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [73]:
prediction = tree.predict(X_test) # threshold에 따라서 predict 달라진다

In [74]:
prediction_proba = tree.predict_proba(X_test)[:,1] # 확률 구하기 

In [75]:
y_test = pd.DataFrame(y_test)
y_test.columns = ['Actual']

In [76]:
y_test['Predict'] = prediction
y_test['Predict_proba'] = prediction_proba

### feature importance

In [77]:
tree.feature_importances_
# ['fan_per', 'like_per', 'album_score_per', 'album_like_like_per'][0.06908332, 0.13545797, 0.54701395, 0.24844475

array([0.06908332, 0.13545797, 0.54701395, 0.24844475])

### Confusion Matrix

In [78]:
y_test.groupby(['Actual', 'Predict']).size()
# 0 false, 1 true
# 0 nagative, 1 positive

Actual  Predict
0       0          144
        1            7
1       0            3
        1           13
dtype: int64